In [27]:
def getDict(path = 'dataset/행정표준용어.xls') :
    # convert dataframe_to_dictionary
    # key = 용어명, value = [용어영문명, 영문약어명, 정의]
    import pandas as pd
    tmp = pd.read_excel(path, header=1, index_col=0)
    
    newDict = {}
    for idx in tmp.index :
        newDict[tmp.at[idx,'용어명']] = [tmp.at[idx,'용어영문명'],tmp.at[idx,'영문약어명'],tmp.at[idx,'정의'] ]
        
    return newDict

In [43]:
# translate_function
def translate_by_selenium_naver(text) :
    from selenium import webdriver
    import urllib.request
    import requests
    import time
    from bs4 import BeautifulSoup
    
    chromedriver_dir = 'C:/dev/driver/chromedriver.exe'
    driver = webdriver.Chrome(chromedriver_dir)
    
    driver.get("https://papago.naver.com/")
    driver.implicitly_wait(3)
    driver.find_element_by_id('txtSource').send_keys(text)
    
    time.sleep(1)
    output_ = driver.find_element_by_id('txtTarget').text
    
    driver.close()
    
    return output_

def translate_by_selenium(text) :
    from selenium import webdriver
    import urllib.request
    import requests
    import time
    from bs4 import BeautifulSoup
    
    chromedriver_dir = 'C:\dev\driver\chromedriver.exe'
    driver = webdriver.Chrome(chromedriver_dir)

    driver.get("https://translate.google.com/")

    driver.find_element_by_id('sugg-item-en').click()
    driver.find_element_by_id('sugg-item-ko').click()

    driver.find_element_by_id('source').send_keys(text)

    time.sleep(1)
    source = driver.page_source
    soupData = BeautifulSoup(source,'html.parser')
    
    driver.close()

    return soupData.find('span',class_='tlid-translation translation').text

def translate_by_papago_api(text) :
    import os
    import sys
    import urllib.request
    import json
    
    client_id = "bONH_8tRvIGnJbn7o966"
    client_secret = "zuoA87y5rv"
    
    output_ = ''
    encText = urllib.parse.quote(text)
    data = "source=ko&target=en&text=" + encText
    url = "https://openapi.naver.com/v1/papago/n2mt"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        str2dict = json.loads(response_body.decode('utf-8'))
        output_ = str2dict['message']['result']['translatedText']
    else:
        output_ = "Error Code:" +rescode

    return output_
    

In [ ]:
# 국어사전 검색
def get_definition(text) :
    from selenium import webdriver
    import urllib.request
    import requests
    import time
    
    chromedriver_dir = 'C:/dev/driver/chromedriver.exe'
    driver = webdriver.Chrome(chromedriver_dir)
    
    driver.get("https://ko.dict.naver.com/#/main")
    
    driver.find_element_by_id('ac_input').send_keys(text)
    
    driver.find_element_by_xpath("//button[@class='btn_search']").click()
    
    time.sleep(1)
    driver.find_element_by_xpath("//div[@class='component_keyword has-saving-function']/div[@class='row']/div[@class='origin']/a").click()
    time.sleep(1)
    
    try :
        output_ = driver.find_element_by_xpath("//ul[@class='mean_list my_mean_list']").text
    except : 
        output_ = driver.find_element_by_xpath("//li[@class='mean_item']/div[@class='mean_desc']/p[@class='cont']").text
    finally :
        driver.close()
    
    return output_


In [ ]:
# make abrv
def changeAbrv(text,origin,dict_) :
    import re

    v = ['a','e','i','o','u']
    tmp = text.lower()[1:]

    for ch in v :
        tmp = tmp.replace(ch,'')

    for ch in tmp:
        dup = ch+ch
        tmp = re.sub(dup, ch, tmp)
    
    tmp = re.sub(' ','',tmp)
    tmp = (text[0].lower() + tmp).upper()
    
    result = ''
    isFound = False
    
    for idx in range(3,len(tmp)) :
        if not tmp[:idx] in dict_.values() :
            result = tmp[:idx]
            isFound = True
            break
    
    # 마지막까지 단어가 있으면 '_' 추가
    if not isFound :
        if not tmp in dict_.values() :
            result = tmp
        else :
            result = tmp + '_'
            
    # dict updated
    dict_[origin] = result
    
    return result,dict_

In [ ]:
def convertFile_v2(inputDict, input_path,output_path='output') :
    import pandas as pd
    
    input_ = pd.read_excel(input_path)
    if output_path[input_path.rfind('/'):].find('.') == -1 :
        output_path += input_path[input_path.rfind('/'):]

    defDict = getDataDict()
    newDict = {}
    
    for idx in input_['output'].isnull().index :
        tmp, inputDict,defDict,newDict = convert_v4(input_.at[idx,'input'],inputDict,defDict,newDict)
        input_.output[idx] = tmp
        
    input_.to_excel(output_path,index=False)
    outputDict(newDict,output_path[:output_path.rfind('/')]+'/newDict.xlsx')    

In [28]:
# class spacing_konlpy()
def getNouns(input_) :
    from konlpy.tag import Okt
    tl = Okt().pos(input_)
    rl = []
    tmp =''

    for idx in range(len(tl)) :    
        if tl[idx][1] == 'Noun' :
            if tmp != '' :
                rl.append(tmp)
            rl.append(tl[idx][0])
            tmp = ''
        else :
            tmp += str(tl[idx][0])
    
    if tmp != '' :
        rl.append(tmp)
            
    return rl

In [30]:
def select_spacing(type_, input_) :
    if type_.lower() == 'konlpy' :
        return getNouns(input_)
#     elif type_.lower() == 'myspacing' :
#         return spacing
    else :
        raise print('error')

In [ ]:
def select_spacing(type_, input_) :
    if type_ == 'konlpy' :
        return getNouns(input_)
#     elif type_ == 'myspacing' :
#         return spacing
    else :
        raise print('error')

def convert(input_,inputDict,defDict,newDict) :
    result = ''
    if input_ != [] :
        for idx in range(len(input_),0,-1) :
            tmpTxt = sumStrList(list_[:idx])
            if tmpTxt in inputDict.keys() :
                result += inputDict[tmpTxt] + '_'
                if idx != len(input_) :
                    resTmp,inputDict,defDict,newDict = convert(sumStrList(input_[idx:]),inputDict,
                                                                  defDict,newDict)
                    result += resTmp
                break
            elif tmpTxt in defDict.keys() :
                result += defDict[tmpTxt] + '_'
                if idx != len(input_) :
                    resTmp,inputDict,defDict,newDict = convert(sumStrList(input_[idx:]),inputDict,
                                                                  defDict,newDict)
                    result += resTmp
                break
            elif tmpTxt in newDict.keys() :
                result += newDict[tmpTxt] + '_'
                if idx != len(input_) :
                    resTmp,inputDict,defDict,newDict = convert(sumStrList(input_[idx:]),inputDict,
                                                                  defDict,newDict)
                    result += resTmp
                break

            if idx == 1 :
                tmp,newDict = changeAbrv_v2(translate_by_papago_api(tmpTxt),tmpTxt,newDict)
                result += tmp + '_'
                if idx != len(input_) :
                    resTmp,inputDict,defDict,newDict = convert(sumStrList(input_[idx:]),inputDict,
                                                                  defDict,newDict)
                    result += resTmp
                    
                    
        if result[-1] == '_' :
            result = result[:-1]
    
    return result, inputDict, defDict, newDict